In [1]:
import pandas as pd
import geopandas as gpd
import openrouteservice as ors
import folium
import sys
import concurrent.futures
from tqdm import tqdm
from threading import Semaphore
import json
import folium
import openrouteservice
import json
import time
import openrouteservice
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely.geometry import shape
from geopy.distance import geodesic
from shapely import wkt
from openrouteservice.exceptions import ApiError
sys.path.append('../scripts')
from utilities import get_vic_subs_as_csv

In [2]:
client = ors.Client(key='5b3ce3597851110001cf624828f2b63314b24af798ed8608d8464d19')

In [3]:
df = pd.read_csv('../data/raw/public_hospital_list.csv')

# Filter the DataFrame for rows where the State is 'VIC'
vic_hospitals = df[df['State'] == 'VIC'].reset_index(drop=True)

# Display the filtered DataFrame
print(vic_hospitals)

    State                           Hospital name Establishment ID  \
0     VIC   Albury Wodonga Health [Albury Campus]        210N01650   
1     VIC  Albury Wodonga Health [Wodonga Campus]        210401660   
2     VIC             Alexandra District Hospital        210403010   
3     VIC                  Alpine Health [Bright]        210401882   
4     VIC            Alpine Health [Mount Beauty]        210401883   
..    ...                                     ...              ...   
147   VIC                   Womens at Sandringham        210A01232   
148   VIC                     Yarra Ranges Health        210803690   
149   VIC        Yarram & District Health Service        210501770   
150   VIC      Yarrawonga District Health Service        210401780   
151   VIC        Yea & District Memorial Hospital        210403590   

    Medicare Provider No.  Local Hospital Network identifier  \
0                0010560W                                249   
1                0031390A      

In [4]:
def get_coordinates(hospital_name):
    try:
        response = client.pelias_search(hospital_name)
        if response['features']:
            # Get the first result's coordinates
            coords = response['features'][0]['geometry']['coordinates']
            return coords  # Returns [longitude, latitude]
        else:
            return None
    except Exception as e:
        print(f"Error retrieving coordinates for {hospital_name}: {e}")
        return None
    
coordinates = []

for index, row in vic_hospitals.iterrows():
    coords = get_coordinates(row['Hospital name'])
    coordinates.append(coords)
    
    time.sleep(0.4)
    if (index + 1) % 20 == 0:
        print(f"Processed {index + 1} hospitals...")

Processed 20 hospitals...
Processed 40 hospitals...
Processed 60 hospitals...
Processed 80 hospitals...
Processed 100 hospitals...
Processed 120 hospitals...
Processed 140 hospitals...


In [15]:
vic_hospitals['geometry'] = coordinates
vic_hospitals.at[10, 'geometry'] = [143.84332896825458, -37.56401250975259]
vic_hospitals.at[39, 'geometry'] = [144.43449191243988, -37.67782265184668]
vic_hospitals.at[59, 'geometry'] = [143.9855122817767, -38.24337478257656]
vic_hospitals.at[140, 'geometry'] = [141.69262555076315, -37.59981379485332]
vic_hospitals.at[141, 'geometry'] = [142.030389056623, -37.73705834823031]
vic_hospitals.at[142, 'geometry'] = [142.28209656248157, -37.87711339669303]
vic_hospitals.to_csv('../data/raw/vic_hospitals_with_coordinates.csv', index=False)

print("Coordinates added and saved to vic_hospitals_with_coordinates.csv")


Coordinates added and saved to vic_hospitals_with_coordinates.csv


In [16]:
type(vic_hospitals.iloc[59]['geometry'])

list

In [24]:
hospital_coordinates = []

for index, row in vic_hospitals.iterrows():
    geometry = row['geometry']
    
    # Check if geometry is a list and has the correct length
    if isinstance(geometry, list) and len(geometry) == 2:
        longitude = geometry[0]
        latitude = geometry[1]
        hospital_coordinates.append({
            'name': row['Hospital name'],
            'coordinates': (latitude, longitude)
        })
    else:
        print(f"Invalid geometry format for hospital: {row['Hospital name']}, geometry: {geometry}")


In [25]:
print(hospital_coordinates)

[{'name': 'Albury Wodonga Health [Albury Campus]', 'coordinates': (-36.074771, 146.914504)}, {'name': 'Albury Wodonga Health [Wodonga Campus]', 'coordinates': (-36.1039, 146.90042)}, {'name': 'Alexandra District Hospital', 'coordinates': (32.833162, -96.951322)}, {'name': 'Alpine Health [Bright]', 'coordinates': (-36.726793, 146.964284)}, {'name': 'Alpine Health [Mount Beauty]', 'coordinates': (-36.7436, 147.169727)}, {'name': 'Alpine Health [Myrtleford]', 'coordinates': (-36.554277, 146.714166)}, {'name': 'Angliss Hospital', 'coordinates': (-37.898645, 145.314467)}, {'name': 'Austin Hospital [Heidelberg]', 'coordinates': (-37.753927, 145.047895)}, {'name': 'Bairnsdale Regional Health Service', 'coordinates': (-37.831508, 147.607868)}, {'name': 'Ballarat Health Services [Base Campus]', 'coordinates': (35.054623, -106.622997)}, {'name': 'Ballarat Health Services [Queen Elizabet', 'coordinates': (-37.56401250975259, 143.84332896825458)}, {'name': 'Bass Coast Regional Health', 'coordinate

In [26]:
with open('../data/raw/houses_with_stations.json') as f:
    houses_data = json.load(f)

In [27]:
def find_closest_hospital(house_coordinates):
    closest_hospital = None
    min_distance = float('inf')
    
    for hospital in hospital_coordinates:
        distance = geodesic(house_coordinates, hospital['coordinates']).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_hospital = {
                'name': hospital['name'],
                'distance_km': min_distance,
                'coordinates': hospital['coordinates']
            }
    
    return closest_hospital

In [28]:
for url, house_info in houses_data.items():
    house_coords = (house_info['coordinates']['latitude'], house_info['coordinates']['longitude'])
    # print(house_coords)
    closest_hospital = find_closest_hospital(house_coords)
    house_info['closest_hospital'] = closest_hospital

# Save the updated houses data with closest hospital information
# with open('data/raw/houses_with_stations.json', 'w') as f:
#     json.dump(houses_data, f, indent=4)

# print("Closest hospitals added and saved to houses_with_stations.json")

In [30]:
#  Save the updated houses data with closest hospital information
with open('../data/raw/houses_with_stations.json', 'w') as f:
    json.dump(houses_data, f, indent=4)

print("Closest hospitals added and saved to houses_with_stations.json")

Closest hospitals added and saved to houses_with_stations.json
